In [1]:
import os
import shutil
import json
import sigopt
import math
import torch
import random
import numpy as np
import pandas as pd
from processing.dataloader.dataloader import get_dataloader
from processing.utils import filter_data_by_properties,select_structures
from training.evaluate import evaluate_model
from processing.interpolation.Interpolation import *
from training.loss import contrastive_loss
from training.sigopt_utils import build_sigopt_name
from processing.create_model.create_model import create_model

saved_models_path = "/home/jypeng/publication_repos/PerovskiteOrderingGCNNs/saved_models/"


def get_experiment_id_fix_painn(model_params, target_prop):
    
    f = open('inference/experiment_ids.json')
    settings_to_id = json.load(f)
    f.close()

    sigopt_name = build_sigopt_name(model_params["data"], target_prop, model_params["struct_type"], model_params["interpolation"], model_params["model_type"],contrastive_weight=model_params["contrastive_weight"],training_fraction=model_params["training_fraction"],long_range=model_params["long_range"])
    print(sigopt_name)

    if sigopt_name in settings_to_id:
        return settings_to_id[sigopt_name]
    else:
        raise ValueError('These model parameters have not been studied')


def load_model_fix_painn(gpu_num, train_loader, target_prop, model_params, folder_idx, job_idx,per_site):
    device_name = "cuda:" + str(gpu_num)
    device = torch.device(device_name)
    
    sigopt_name = build_sigopt_name(model_params["data"], target_prop, model_params["struct_type"], model_params["interpolation"], model_params["model_type"],contrastive_weight=model_params["contrastive_weight"],training_fraction=model_params["training_fraction"],long_range=model_params["long_range"])
    exp_id = get_experiment_id_fix_painn(model_params, target_prop)
    directory = saved_models_path + model_params["model_type"] + "/"+ sigopt_name + "/" +str(exp_id)+"/" + "observ_" + str(folder_idx)

    if model_params["model_type"] == "Painn":
        model = torch.load(directory + "/best_model", map_location=device)
        normalizer = None
    # else:
    #     model, normalizer = create_model(model_params["model_type"], train_loader,model_params["interpolation"],target_prop,hyperparameters=assignments,per_site=per_site)
    #     model.to(device)
    #     model.load_state_dict(torch.load(directory + "/best_model.torch", map_location=device)['state'])
    
    return model, normalizer


def reverify_sigopt_models_fix_painn(model_params, gpu_num, target_prop):
    device_name = "cuda:" + str(gpu_num)
    device = torch.device(device_name)
    torch.cuda.set_device(device)
    
    interpolation = model_params["interpolation"]
    model_type = model_params["model_type"]    
    data_name = model_params["data"]
    struct_type = model_params["struct_type"]

    if data_name == "data/":

        training_data = pd.read_json(data_name + 'training_set.json')
        training_data = training_data.sample(frac=model_params["training_fraction"],replace=False,random_state=0)
        validation_data = pd.read_json(data_name + 'validation_set.json')
        edge_data = pd.read_json(data_name + 'edge_dataset.json')

        if not interpolation:
            training_data = pd.concat((training_data,edge_data))

    else:
        print("Specified Data Directory Does Not Exist!")


    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

    print("Loaded data")

    data = [training_data, validation_data]
    processed_data = []

    for dataset in data:
        dataset = filter_data_by_properties(dataset,target_prop)

        dataset = select_structures(dataset,struct_type)

        processed_data.append(dataset)

    print("Completed data processing")
    
    train_data = processed_data[0]
    validation_data = processed_data[1]
    
    per_site = False

    train_loader = get_dataloader(train_data,target_prop,model_type,1,interpolation,per_site=per_site,long_range=model_params["long_range"])
    val_loader = get_dataloader(validation_data,target_prop,model_type,1,interpolation,per_site=per_site,long_range=model_params["long_range"])       

    reverify_sigopt_models_results = pd.DataFrame(columns=['sigopt_loss', 'reverified_loss'])

    exp_id = get_experiment_id_fix_painn(model_params, target_prop)
    
    for folder_idx in range(16):        
        job_idx = 16 - folder_idx - 1
        print('Reverifying sigopt model #' + str(folder_idx))
        
        sigopt_name = build_sigopt_name(model_params["data"], target_prop, model_params["struct_type"], model_params["interpolation"], model_params["model_type"],contrastive_weight=model_params["contrastive_weight"],training_fraction=model_params["training_fraction"],long_range=model_params["long_range"])
        directory = saved_models_path + model_params["model_type"] + "/"+ sigopt_name + "/" +str(exp_id)+"/" + "observ_" + str(folder_idx)

        model, normalizer = load_model_fix_painn(gpu_num, train_loader, target_prop, model_params, folder_idx, job_idx, per_site=per_site)
        
        
        if "contrastive" in model_type:
            loss_fn = contrastive_loss 
            is_contrast = True
            
        else:
            loss_fn = torch.nn.L1Loss()
            is_contrast = False
            
        _, _, best_loss = evaluate_model(model, normalizer, model_type, val_loader, loss_fn, gpu_num,is_contrastive=is_contrast,contrastive_weight=model_params["contrastive_weight"])


        if model_type == "Painn":
            reverified_loss = best_loss
        else:
            reverified_loss = best_loss[0]

        new_row = pd.DataFrame([[reverified_loss, reverified_loss]], columns=['sigopt_loss', 'reverified_loss'])

        reverify_sigopt_models_results = pd.concat([
            reverify_sigopt_models_results,
            new_row
        ], ignore_index=True)

    sigopt_name = build_sigopt_name(model_params["data"], target_prop, model_params["struct_type"], model_params["interpolation"], model_params["model_type"],contrastive_weight=model_params["contrastive_weight"],training_fraction=model_params["training_fraction"],long_range=model_params["long_range"])
    exp_id = get_experiment_id_fix_painn(model_params, target_prop)
    save_directory = saved_models_path + model_params["model_type"] + "/"+ sigopt_name + "/" +str(exp_id)
    reverify_sigopt_models_results.to_csv(save_directory + "/reverify_sigopt_models_results.csv")


def keep_the_best_few_models_fix_painn(model_params, target_prop, num_best_models=3):
    sigopt_name = build_sigopt_name(model_params["data"], target_prop, model_params["struct_type"], model_params["interpolation"], model_params["model_type"],contrastive_weight=model_params["contrastive_weight"],training_fraction=model_params["training_fraction"],long_range=model_params["long_range"])
    exp_id = get_experiment_id_fix_painn(model_params, target_prop)
    old_directory_prefix = saved_models_path + model_params["model_type"] + "/"+ sigopt_name + "/" +str(exp_id)
    new_directory_prefix = "./best_models/" + model_params["model_type"] + "/"+ sigopt_name + "/" +str(exp_id)

    reverify_sigopt_models_results = pd.read_csv(old_directory_prefix + '/reverify_sigopt_models_results.csv', index_col=0)

    for index, row in reverify_sigopt_models_results.iterrows():
        if not math.isclose(row['sigopt_loss'], row['reverified_loss'], rel_tol=1e-2):
            print("========\nSigopt model #" + str(index) + " has non-matching losses\n========")
    
    for i in range(num_best_models):
        new_directory = new_directory_prefix + "/best_" + str(i)
        if not os.path.exists(new_directory):
            os.makedirs(new_directory)

        folder_idx = reverify_sigopt_models_results.sort_values(by=['sigopt_loss']).index[i]
        old_directory = old_directory_prefix + "/observ_" + str(folder_idx)

        if model_params['model_type'] == "Painn":
            file_name = "best_model"
        else:
            file_name = "best_model.torch"

        shutil.copy(old_directory + "/" + file_name, new_directory + "/" + file_name)
        # shutil.copy(old_directory + "/hyperparameters.json", new_directory + "/hyperparameters.json")
        print("Best models #" + str(i) + " found and saved")

/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "Painn",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":False
}

In [3]:
reverify_sigopt_models_fix_painn(model_params, gpu_num, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 11187.40it/s]


dft_e_hull_htvs_data_unrelaxed_Painn
Reverifying sigopt model #0
dft_e_hull_htvs_data_unrelaxed_Painn


100%|██████████| 1277/1277 [00:10<00:00, 124.99it/s]


Reverifying sigopt model #1
dft_e_hull_htvs_data_unrelaxed_Painn


...

Reverifying sigopt model #14
dft_e_hull_htvs_data_unrelaxed_Painn


100%|██████████| 1277/1277 [00:05<00:00, 227.84it/s]


Reverifying sigopt model #15
dft_e_hull_htvs_data_unrelaxed_Painn


100%|██████████| 1277/1277 [00:08<00:00, 153.62it/s]


dft_e_hull_htvs_data_unrelaxed_Painn


In [4]:
keep_the_best_few_models_fix_painn(model_params, target_prop)

dft_e_hull_htvs_data_unrelaxed_Painn
Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [ ]:
# Previous codes

from inference.select_best_models import *
# tar cf best_models.tar best_models
# tar cf saved_models.tar saved_models

In [2]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1277/1277 [00:00<00:00, 3986.62it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [02:46<00:00, 37.78it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 9382.02it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 9703.96it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [3]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1277/1277 [00:00<00:00, 3500.65it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [02:45<00:00, 38.00it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 9588.49it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 9642.47it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [4]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1277/1277 [00:00<00:00, 3943.30it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [02:44<00:00, 38.08it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 7110.61it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 7199.96it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1277/1277 [00:00<00:00, 3968.12it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [02:46<00:00, 37.70it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 9752.21it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #45


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6276/6276 [00:00<00:00, 9666.55it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #46


 60%|████████████████████████████████████████████████████████▋                                      | 3744/6276 [00:00<00:00, 9846.23it/s]

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "Painn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "Painn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "M3Gnet_relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [2]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:38<00:00, 33.26it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [3]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:38<00:00, 33.02it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [4]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:39<00:00, 32.28it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [5]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 3954.36it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 4467/4467 [02:07<00:00, 34.96it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 4467/4467 [00:00<00:00, 9156.57it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 4467/4467 [00:00<00:00, 9059.80it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [6]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 3938.96it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 3563/3563 [01:40<00:00, 35.34it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 3563/3563 [00:00<00:00, 9231.66it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 3563/3563 [00:00<00:00, 9034.55it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [7]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:39<00:00, 32.36it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [8]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:40<00:00, 31.91it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [9]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:39<00:00, 32.13it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [10]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 3453.76it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 4467/4467 [02:05<00:00, 35.63it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 4467/4467 [00:00<00:00, 9349.02it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 4467/4467 [00:00<00:00, 10054.32it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [11]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 3317.53it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 3563/3563 [01:39<00:00, 35.73it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 3563/3563 [00:00<00:00, 8620.09it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 3563/3563 [00:00<00:00, 7995.82it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [12]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 3486.68it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 3111/3111 [01:25<00:00, 36.30it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 3111/3111 [00:00<00:00, 8439.21it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 3111/3111 [00:00<00:00, 8138.35it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [13]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:40<00:00, 31.22it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [14]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:41<00:00, 30.80it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [15]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:42<00:00, 30.25it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [16]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 3762.17it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 4467/4467 [02:08<00:00, 34.81it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 4467/4467 [00:00<00:00, 7074.86it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 4467/4467 [00:00<00:00, 7587.24it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [17]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 3736.10it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 3563/3563 [01:48<00:00, 32.94it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 3563/3563 [00:00<00:00, 9120.82it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 3563/3563 [00:00<00:00, 9042.24it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:39<00:00, 31.93it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #3


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #4


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.5
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.25
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [46]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1064/1064 [00:32<00:00, 33.03it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [47]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1064/1064 [00:32<00:00, 32.69it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [48]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1064/1064 [00:00<00:00, 3409.28it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 2886/2886 [01:25<00:00, 33.64it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 2886/2886 [00:00<00:00, 7806.81it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 2886/2886 [00:00<00:00, 8153.83it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [49]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1064/1064 [00:00<00:00, 3931.30it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 2886/2886 [01:24<00:00, 34.11it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 2886/2886 [00:00<00:00, 8326.41it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 2886/2886 [00:00<00:00, 8359.24it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [50]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1065/1065 [00:32<00:00, 32.93it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [51]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "e3nn",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1065/1065 [00:31<00:00, 33.55it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [52]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1065/1065 [00:00<00:00, 3398.70it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 2887/2887 [01:27<00:00, 32.83it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 2887/2887 [00:00<00:00, 6776.42it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 2887/2887 [00:00<00:00, 6917.92it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [53]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": True,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1065/1065 [00:00<00:00, 3916.25it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 2887/2887 [01:28<00:00, 32.50it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 2887/2887 [00:00<00:00, 8586.71it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 2887/2887 [00:00<00:00, 8739.37it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [54]:
gpu_num=0
target_prop="Op_band_center"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 4040.20it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 3111/3111 [01:15<00:00, 41.37it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 3111/3111 [00:00<00:00, 9259.93it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 3111/3111 [00:00<00:00, 9282.46it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [56]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":0.125
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████| 1277/1277 [00:00<00:00, 4018.75it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|██████████| 3111/3111 [01:17<00:00, 40.33it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████| 3111/3111 [00:00<00:00, 9186.47it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████| 3111/3111 [00:00<00:00, 9146.47it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":True
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=0
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "CGCNN",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":True
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":False
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


 36%|███▌      | 2250/6276 [21:34<37:07,  1.81it/s]  

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "relaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":0.1,
    "training_fraction":1.0,
    "long_range":False
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":1.0,
    "training_fraction":1.0,
    "long_range":False
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":0.1,
    "training_fraction":1.0,
    "long_range":False
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


 15%|█▍        | 924/6276 [08:50<46:51,  1.90it/s]  

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":10.0,
    "training_fraction":1.0,
    "long_range":False
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

In [ ]:
gpu_num=1
target_prop="dft_e_hull"

model_params = {
    "data":"data/",
    "struct_type": "unrelaxed",
    "interpolation": False,
    "model_type": "e3nn_contrastive",
    "contrastive_weight":100.0,
    "training_fraction":1.0,
    "long_range":False
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)